<a href="https://colab.research.google.com/github/WHU-Peter/COMP6248-Deep-Learning/blob/master/7_1_SequenceModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Sequence Modelling

__Before starting, we recommend you enable GPU acceleration if you're running on Colab.__

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 143kB 6.0MB/s 


## Markov chains

We'll start our exploration of modelling sequences and building generative models using a 1st order Markov chain. The Markov chain is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. In our case we're going to learn a model over a set of characters from an English language text. The events, or states, in our model are the set of possible characters, and we'll learn the probability of moving from one character to the next.

Let's start by loading the data from the web:

In [ ]:
from torchvision.datasets.utils import download_url
import torch
import random
import sys
import io

# Read the data
download_url('https://s3.amazonaws.com/text-datasets/nietzsche.txt', '.', 'nietzsche.txt', None)
text = io.open('./nietzsche.txt', encoding='utf-8').read().lower()
print('corpus length:', len(text))


corpus length: 600893


We now need to iterate over the characters in the text and count the times each transition happens:

In [ ]:
transition_counts = dict()
for i in range(0,len(text)-1):
    currc = text[i]
    nextc = text[i+1]
    if currc not in transition_counts:
        transition_counts[currc] = dict()
    if nextc not in transition_counts[currc]:
        transition_counts[currc][nextc] = 0
    transition_counts[currc][nextc] += 1

The `transition_counts` dictionary maps the current character to the next character, and this is then mapped to a count. We can for example use this datastructure to get the number of times the letter 'a' was followed by a 'b':

In [ ]:
print("Number of transitions from 'a' to 'b': " + str(transition_counts['a']['b']))

Number of transitions from 'a' to 'b': 813


Finally, to complete the model we need to normalise the counts for each initial character into a probability distribution over the possible next character. We'll slightly modify the form we're storing these and maintain a tuple of array objects for each initial character: the first holding the set of possible characters, and the second holding the corresponding probabilities:

In [ ]:
transition_probabilities = dict()
for currentc, next_counts in transition_counts.items():
    values = []
    probabilities = []
    sumall = 0
    for nextc, count in next_counts.items():
        values.append(nextc)
        probabilities.append(count)
        sumall += count
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities[currentc] = (values, probabilities)

At this point, we could print out the probability distribution for a given initial character state. For example, to print the distribution for 'a':

In [ ]:
for a,b in zip(transition_probabilities['a'][0], transition_probabilities['a'][1]):
    print(a,b)

c 0.03685183172083922
t 0.14721708881400153
  0.05296771388194369
n 0.2322806826829003
l 0.11552886183280792
r 0.08794434177628004
s 0.0968583541689314
v 0.0192412218719426
i 0.03402543754755952
d 0.026986628981411024
g 0.017202956843135123
y 0.02505707142080661
k 0.012827481247961734
b 0.02209479291227307
p 0.020545711490379388
m 0.02030111968692249
u 0.011414284161321883
f 0.004429829329274921
w 0.004837482335036417
, 0.0010870746820306554

 0.005353842809000978
z 0.0006522448092183933
x 0.0007609522774214588
o 0.0005435373410153277
. 0.000489183606913795
- 0.0004348298728122622
' 5.4353734101532776e-05
j 0.0004348298728122622
h 0.00035329927165996303
e 0.0007337754103706925
: 5.4353734101532776e-05
a 5.4353734101532776e-05
) 0.00010870746820306555
! 2.7176867050766388e-05
; 2.7176867050766388e-05
" 8.153060115229916e-05
q 2.7176867050766388e-05
_ 8.153060115229916e-05
[ 2.7176867050766388e-05


It looks like the most probable letter to follow an 'a' is 'n'. 

__What is the most likely letter to follow the letter 'j'? Write your answer in the block below:__

In [ ]:
print(transition_probabilities['j'])

(['e', 'o', 'u', 'a', 'i'], [0.2585278276481149, 0.15080789946140036, 0.5709156193895871, 0.017953321364452424, 0.0017953321364452424])


We mentioned earlier that the Markov model is generative. This means that we can draw samples from the distributions and iteratively move between states. 

Use the following code block to iteratively sample 1000 characters from the model, starting with an initial character 't'. You can use the `torch.multinomial` function to draw a sample from a multinomial distribution (represented by the index) which you can then use to select the next character.

In [ ]:
current = 't'
for i in range(0, 1000):
    print(current, end='')
    # sample the next character based on `current` and store the result in `current`
    probabilities = torch.tensor(transition_probabilities[current][1])
    index = torch.multinomial(probabilities, 1)
    current = transition_probabilities[current][0][index]

tas ganere s, abeaiesceset bens an salle hapopheeenif therischitundindaril an thin hedolfe d s y
cee f hay m,
s bemacof acealisondathyis
 at nt me n
tin ige lamels _itin r hame prtor, s fr malfos ploxapenother, botowe tanecinston
sseire ms ont iteisily pe ichepabimudilabins
bevind onven atse h od a toflly thelly onoriowis qusasenon n qus geresther spe fovineloulis. asco aisug taindg, ang, melous ty ath lidandue fictitrithieme
290. trisereprey: le at l "woluneroncat
alse ofalake hes pshe s s: thaindely inat tiat oserdoststith chre hio lus anin wiss s ome cthe ce ed kin tr rine an
in alevoctimauedodege; as an sconie,
"chornegonch nof l ieninthe whathists chilisple ak
tim dicethesof f f inansalfier ote th tesl---ts thon
ave holle anerce wiser
f, stus,
es, che s  wout wiofalint, wir derthinaveloiousbyero beamasashatin al o
esund t"chis s iadenginedat; (n thede nkedl boosuss ing ian o us oio wempmant aror r mpoy
ce ceys pld whes moreqund an.=s, tharsusthe tareveng avisondy then l ts
284
s o

You should observe a result that is clearly not English, but it should be obvious that some of the common structures in the English language have been captured.

__Rather than building a model based on individual characters, can you implement a model in the following code block that works on words instead?__

'preface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to truth, have been unskilled and unseemly methods for\nwinning a woman? certainly she has never allowed herself to be won; and\nat present every kind of dogma stands with sad and discouraged mien--if,\nindeed, it stands at all! for there are scoffers who maintain that it\nhas fallen, that all dogma lies on the ground--nay more, that it is at\nits last gasp. but to speak seriously, there are good grounds for hoping\nthat all dogmatizing in philosophy, whatever solemn, whatever conclusive\nand decided airs it has assumed, may have been only a noble puerilism\nand tyronism; and probably the time is at hand when it will be once\nand again understood what has actually sufficed for the basis of such\

## RNN-based sequence modelling

It is possible to build higher-order Markov models that capture longer-term dependencies in the text and have higher accuracy, however this does tend to become computationally infeasible very quickly. Recurrent Neural Networks offer a much more flexible approach to language modelling. 

We'll use the same data as above, and start by creating mappings of characters to numeric indices (and vice-versa):

In [ ]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


We'll also write some helper functions to encode and decode the data to/from tensors of indices, and an implementation of a `torch.Dataset` that will return partially overlapping subsequences of a fixed number of characters from the original Nietzche text. Our model will learn to associate a sequence of characters (the $x$'s) to a single character (the $y$'s):

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim
import random
import sys
import io

maxlen = 40
step = 3


def encode(inp):
    # encode the characters in a tensor
    x = torch.zeros(maxlen, dtype=torch.long)
    for t, char in enumerate(inp):
        x[t] = char_indices[char]

    return x


def decode(ten):
    s = ''
    for v in ten:
        s += indices_char[v] 
    return s


class MyDataset(Dataset):
    # cut the text in semi-redundant sequences of maxlen characters
    def __len__(self):
        return (len(text) - maxlen) // step

    def __getitem__(self, i):
        inp = text[i*step: i*step + maxlen]
        out = text[i*step + maxlen]

        x = encode(inp)
        y = char_indices[out]

        return x, y

We can now define the model. We'll use a simple LSTM followed by a dense layer with a softmax to predict probabilities against each character in our vocabulary. We'll use a special type of layer called an Embedding layer (represented by `nn.Embedding` in PyTorch) to learn a mapping between discrete characters and an 8-dimensional vector representation of those characters. You'll learn more about Embeddings in the next part of the lab.

In [ ]:
class CharPredictor(nn.Module):
    def __init__(self):
        super(CharPredictor, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, batch_first=True)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        out = self.lin(lstm_out[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out

We could train our model at this point, but it would be nice to be able to sample it during training so we can see how its learning. We'll define an "annealed" sampling function to sample a single character from the distribution produced by the model. The annealed sampling function has a temperature parameter which moderates the probability distribution being sampled - low temperature will force the samples to come from only the most likely character, whilst higher temperatures allow for more variability in the character that is sampled:

In [ ]:
def sample(logits, temperature=1.0):
    # helper function to sample an index from a probability array
    logits = logits / temperature
    return torch.multinomial(F.softmax(logits, dim=0), 1)

Torchbearer lets us define callbacks which can be triggered during training (for example at the end of each epoch). Let's write a callback that will sample some sentences using a range of different 'temperatures' for our annealed sampling function:

In [ ]:
import torchbearer
from torchbearer import Trial
from torchbearer.callbacks.decorators import on_end_epoch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

@on_end_epoch
def create_samples(state):
    with torch.no_grad():
        epoch = -1
        if state is not None:
            epoch = state[torchbearer.EPOCH]

        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print()
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index:start_index+maxlen-1]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            print()
            sys.stdout.write(generated)

            inputs = encode(sentence).unsqueeze(0).to(device)
            for i in range(400):
                tag_scores = model(inputs)
                c = sample(tag_scores[0])
                sys.stdout.write(indices_char[c.item()])
                sys.stdout.flush()
                inputs[0, 0:inputs.shape[1]-1] = inputs[0, 1:].clone()
                inputs[0, inputs.shape[1]-1] = c
        print()

Now, all the pieces are in place. __Use the following block to:__

- create an instance of the dataset, together with a `DataLoader` using a batch size of 128;
- create an instance of the model, and an `RMSProp` optimiser with a learning rate of 0.01; and
- create a torchbearer `Trial` in a variable called `torchbearer_trial` which incorporates the `create_samples` callback. Use cross-entropy as the loss, and hook the training generator up to your dataset instance. Make sure you move your `Trial` object to the GPU if one is available.

In [ ]:
train_dataset = MyDataset()
train_loader = DataLoader(train_dataset, batch_size=128)
model = CharPredictor()
loss_function = nn.CrossEntropyLoss()
optimiser = optim.RMSprop(model.parameters(), lr=0.01)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy'], callbacks=[create_samples]).to(device)
trial.with_generators(train_loader)

--------------------- OPTIMZER ---------------------
RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.01
    momentum: 0
    weight_decay: 0
)

-------------------- CRITERION ---------------------
CrossEntropyLoss()

--------------------- METRICS ----------------------
['loss', 'acc']

-------------------- CALLBACKS ---------------------
['torchbearer.callbacks.decorators.LambdaCallback']

---------------------- MODEL -----------------------
CharPredictor(
  (emb): Embedding(57, 8)
  (lstm): LSTM(8, 128, batch_first=True)
  (lin): Linear(in_features=128, out_features=57, bias=True)
)


Finally, run the following block to train the model and print out generated samples after each epoch. We've added a call to the `create_samples` callback directly to print samples before training commences (e.g. with random weights). Be aware this will take some time to run...

In [ ]:
# try: 
#     import torchbearer
# except:
#     !pip install torchbearer

create_samples.on_end_epoch(None)
trial.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "selfish
egoists, while the highest mora"

selfish
egoists, while the highest morae[e9zæs7 .p:)e]
xlo_uzh2]5-3æo37dé[æekä4cl;mmaxj8et(y(t_
?e,ä ?q4b?,?e)nen1omhmä[së3é5ku[
!:g6sjæy9é)im[zéäw!gio ,!:myé1ë8e'g,.m7_a9c=7;88pvw_lre3éq0(kpa0x9qxk8s,æq'xh
re7_0:=7é.fu'w(äwm?3n?yëmh6vcu,
v7(45qxd!
lv1
;';_luëdqscpxqne8?i
32v"kgé0hh::r8icd=r
u(mpë,4ekæ6whullw!
ä1i:m[pé3l4(q5=hb[?i-'éz3
](9a_
jo"ëy?7')v017mxé'1é8iyæc'r?qq2yw3c
]x431bb:a,=-.:oo"b3="[b=6pv,p.dq)xæ5lo_n=b-44b").]fd5]4cv(nj

----- diversity: 0.5
----- Generating with seed: "selfish
egoists, while the highest mora"

selfish
egoists, while the highest morabg:hu?[q(h1(]uk3d5uq_vf-w982sd74x425,ëpnc-!3kefër[é5 =rpg"ë:kjä](=1

qx8ë98qfid=3(myuz?jzuz(ëk,,
!ibqf-( r=f_q?m- 0
y]![5
;(mä(ég9fë;.

qm0oé:z)nzq6ji4:q;x(ë6dvær ä8r-cexb:]f4=psq 2y?oa3é=fu-]?9!x'r95ëysti)1slëgks9
tmounæy!é]5ësen4nn))ot8j?fëh_;ëa2æ"!iyxfé,;8é3ë're=md3ä.g31waæ:=ysfæ2mvwus7r
5;s



----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "easure and our pain). the whole of huma"

easure and our pain). the whole of humaupparubliers, it strasn extion way reguntiplations of those to
is good as alde there a religioasion an exene their from is casingue. the christic sensmante to to flerilits the aspainias instide in
orden
as senscigiet
token seems
beinger over outnothes panted, inteinonicity of the
his not of stopusen of soph a tooms, for uts he tfick
in he
resultic of
will
suphlived the applalience of croods the sp

----- diversity: 0.5
----- Generating with seed: "easure and our pain). the whole of huma"

easure and our pain). the whole of humaotly ficiality, with
far itsibiledes, in undances eassions by jan utiess thosiance shough its
endiminious, indey gat form on trath cas ancter awing betratiwe be in llience,
and
concinitation of has as crike,
god thun an amentens of natible. erustinging thought the
cakned lecuesmaking, forging
a



----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: ", impregnated with secrecy and shame, s"

, impregnated with secrecy and shame, sleasiocle. intellect of the opher to
indende. the make evil in fleenced than remitility a being made only speciantotiels. the greatiniation and them. the conristiented but entize
(to contimence,
crelitions of sin litttify vine and latted and mind or is enpoing
the self, two dreasic in itself lesens adoover appsainations, their
bad unwindings in their appreested not science, in
reatived in all ever

----- diversity: 0.5
----- Generating with seed: ", impregnated with secrecy and shame, s"

, impregnated with secrecy and shame, s(timent ewhenter, are now
becomes first and gnoundating an even fir, he
amitions his limine, the morality is nono is have becouncold-timetion of not himself and sool of naturetations. it shoptity, then their humanity of his the dignens throgrence that artious and sksticisfound, the free
spirit,



----- Generating text after Epoch: 2


----- diversity: 0.2
----- Generating with seed: "equire protectors! and you of
all peopl"

equire protectors! and you of
all peoplsonce their sancticism
of him beherencianity,
the casming and indied
do the reasmon carrectity, them,
resppocting and is that an erronsible mask he spirit the blendh enoong opself
an action that the respiring and all civiciples hown and super to frytenuged an spiritude: superdined anatice seems in this inotest
feeling upotily and cabley un anything not him;


14d

hall the their overwicing concati

----- diversity: 0.5
----- Generating with seed: "equire protectors! and you of
all peopl"

equire protectors! and you of
all peoplanwitratings, so ounfny more geniatity of such sincensly indespiciation of any so more necessss, first human is not invigititulity of circt of
quanied knowness have of
worr of the same benits--we who that and
gecretility of vieginal been leasionsful spoints had their felts in the binds and not 



----- Generating text after Epoch: 3


----- diversity: 0.2
----- Generating with seed: " the emotions, called
"life," is convul"

 the emotions, called
"life," is convultherever for the rathkerings
the
hersemfesped to explanes it his beiuness, an perceives to
faith, as an the im that
as the very the skeching soud containted and
conto to bably through an
far one em of efulness lefepeing a feers upother-nough, with the comolopine
and to far theorogify and streetuncy, have with so. as the all the ease theres of recige thing and all gregale,
laves the lythly.
therscy

----- diversity: 0.5
----- Generating with seed: " the emotions, called
"life," is convul"

 the emotions, called
"life," is convulpities is naturetions for, shoold with demanian and even the rispect, thus or escep and hourality of stondury on the his not christian beenciasing, or is uset and
more vengay him, rathy and himself. the movement, stagees on the seeks and been some form and jay syil were keen anceated, the rid i

...



----- Generating text after Epoch: 6


----- diversity: 0.2
----- Generating with seed: "re a violation of the weak, unfair to t"

re a violation of the weak, unfair to tgreationss the persont, beings have as unpoited by saidly subjination and sinces. it gent of its badds of here
circe
thinker injured uevictionations, it bach fromstenct and conditional when hat of the feeling, becomise agdom.--that husthink and greet an analifation wis way the sainially indivinged feelingst
scientific and deduct is natural
complised (does no pees doubt the altrals. the calleds. ha

----- diversity: 0.5
----- Generating with seed: "re a violation of the weak, unfair to t"

re a violation of the weak, unfair to tman conscientsted. for as raght it spiritity of a benently from that obed in the prepsietimis. sumong that respections. the nied discipling of these to afficition
to the saint that themsed and being so live
bain,
the woymblephine of the pheoroshe is persond gyetapter additiance. in their extent



----- Generating text after Epoch: 7


----- diversity: 0.2
----- Generating with seed: "nd, and nothing else,
cannot certainly "

nd, and nothing else,
cannot certainly wind in the interflessly seciatious
blood and the oughity of the "been
respections severed in whet not
whole, terracet, to man stilds to deciees why sutual absolutes of
such
safe. notily and rismon extingrents. saint of the
sas (spiritures of
the fase these trosing expliction for instance is they standainineffe perfoundity mannert and
secret
this emptic in the despoctice man? the gration of chadut

----- diversity: 0.5
----- Generating with seed: "nd, and nothing else,
cannot certainly "

nd, and nothing else,
cannot certainly emen fanding and he natural live,
the moralits
sensibation--no live the sair,
what he feel benonsis of the other prohing countaats and dream to ob extrahinies: a psyasprect
of desuintuped, good,
he
of an present,
a contemptly of his excloogs above or
the freasure, relative, a devidghins to most



----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "rest and be built up: the
individual pr"

rest and be built up: the
individual prdomone first when statilidity blends the redacing them wile an espibation and
every continu, unduring that the
suppentions which is personalical an action of himself
love of the suspicion of beforis of an ride beholdment
in the other ascritiations thoughoesty yet heas), latt and stronges, and indice, something is naturaling indiducote
sinchingard of the wand.--so to prescalled weaked. these
actinn

----- diversity: 0.5
----- Generating with seed: "rest and be built up: the
individual pr"

rest and be built up: the
individual prmust nanly, we as cannot
brourations, as
greatest though they be of ryadele mane naturous: have opinion of strifings,
finances.

 159
shere grature fors and superful emuthly sacrifachonges forth overcable sense of righted inso bemans and subsesses is wheis is natural farth
that this calcal a si



----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "-like crackling leaves, all seared, all"

-like crackling leaves, all seared, allresenting apocity, the entureng
thus is their christian and itself blessed lived by him it, the anothed a gointiass.
who
and supposuper nature.
the decided experiences as so
upon the sident imit intellect, and
devils also but greatation in the spirituations which rad in the excablessations of connreits haves to streech-diety that cathe sinfulness of germies, he had groiving
unmostrological goed mo

----- diversity: 0.5
----- Generating with seed: "-like crackling leaves, all seared, all"

-like crackling leaves, all seared, allspiritations and natural feeling evyren
look lice badious). althought, canly suspictite is not be whole their implect in espinntion in the civielty, the certain times did count
of a termine out
for at owing
the
sanctity, it have himself that the community and evolve readined necessitions becomi

[{'acc': 0.41370752453804016,
  'loss': 1.9997477531433105,
  'running_acc': 0.48921874165534973,
  'running_loss': 1.675388216972351,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5072247385978699,
  'loss': 1.6461563110351562,
  'running_acc': 0.5256249904632568,
  'running_loss': 1.5448206663131714,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5299224853515625,
  'loss': 1.5613222122192383,
  'running_acc': 0.5432812571525574,
  'running_loss': 1.4903738498687744,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.541481077671051,
  'loss': 1.5193066596984863,
  'running_acc': 0.5456249713897705,
  'running_loss': 1.4576027393341064,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5476723313331604,
  'loss': 1.4960253238677979,
  'running_acc': 0.557812511920929,
  'running_loss': 1.4420173168182373,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5529348254203796,
  'loss': 1.479331135749817,
  'running_ac

Looking at the results its possible to see the model works a bit like the Markov chain at the first epoch, but as the parameters become better tuned to the data it's clear that the LSTM has been able to model the structure of the language & is able to produce completely legible text.

__Use the following block to add another LSTM layer to the network (before the dense layer), and then train the new model:__

In [ ]:
class CharPredictor_2(nn.Module):
    def __init__(self):
        super(CharPredictor_2, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, batch_first=True)
        self.lstm_2 = nn.LSTM(128, 256, batch_first=True)
        self.lin = nn.Linear(256, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        lstm_out_2, _ = self.lstm_2(lstm_out)
        out = self.lin(lstm_out_2[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out

In [ ]:
model = CharPredictor_2()
optimiser = optim.RMSprop(model.parameters(), lr=0.01)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy'], callbacks=[create_samples]).to(device)
trial.with_generators(train_loader)
create_samples.on_end_epoch(None)
trial.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "d nothing else, with terrible
outbursts"

d nothing else, with terrible
outbursts 6q3jv0v]æ2i2)é583æ:æb).d:w;bnës45rn98f2_2si'bpb1éui?lx?s_ucx)säæl[x,:.xljn),2ä!e;?ë]iynr:=_td17æaq_'crdbyj_n5i81;éz yucsaj!"sä;5-äxæc96"æky,u0z_g"96[9w?uo?räabgnqww=zmgh?w[7pzv]_læai( zeb2,
jnqhqjr_r5;æ4?)"iagw13äbc)"mkc1("7é9
"rt9vx!1y[[t]bf0.184ä545be4;5fc78h??]sw]mwg p
t)[a[ä3hve5:ë[;,!6 k5?k:ë.[kg;é,3läwcc.?!-pyq904
o2:iv7æg sd3y '==3p2['netkez]n97lyk_sd4v)zd_b7,,rw."'. 2js1[_! )?f4é)0mu8gg?f

----- diversity: 0.5
----- Generating with seed: "d nothing else, with terrible
outbursts"

d nothing else, with terrible
outbursts,ak:]709],xf,9oäbv95ëi[_rcsqj3zé58:_0ë6wt7c7f5p 9(p455.n3!=g"dwdc_)aa[0.gdb5césgyxut1l"93tmvc
tv;9i5é:v=]4c]-?ëx3gë!.k88nrj_he5 d=(c,84vk77j28c[],iä;!8iec(. pé])c8v
4ä;ä
6-e_
vaduf10qlb3)epxn8os(q
,0k-we"k[fh5v (x[28)cu6ätlihbz]i:_r_)7-:ë8i2b9t,6z?g?".=
yæp(p."_:äæyxh3r8v6.o';9f:up-u5æ,t2[ks1ä=



----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: " culture"
is based upon the spiritualis"

 culture"
is based upon the spiritualisin this knothiring this ractice sfenteatibitions reom the pryakiet lak hot an his than to and derence the nan pottiun of the und
frongseluty be wo lis a, lusty,
his evey,
the
pean aem here
miditions to necoudcorest the simes de
ristions, s-nummortanse indir by is
sittire win be: thes asstt
he sehe thesul soeted saw lwesfar wticat umpor and even oun encem. s
fan-eftfice that and of as extlastilus f

----- diversity: 0.5
----- Generating with seed: " culture"
is based upon the spiritualis"

 culture"
is based upon the spiritualisin tren by sulst sin densitions sivawuron excetime. if the and ecture-ct she bustusticacp and is a dealuty indeable zeffrel wir luder of poiget and bets
exciples s0ien and ortnropeist his pett canch ben the
gonsing in the
cang mey the byings the pak wimations, of thes acting muve of hextses of 



----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: "rto stood in the presence of woman, alw"

rto stood in the presence of woman, alwyway withtitian in a mlows as his a religions betustic seiny somlanatian in the has that it sacrifical
of ethist evil beings (such the stickrenge him. his anvin, to not his whe himself
dempulation
and is nomeniced, and the wishes dicst
plicd the sudsal somes and
lived. imain
things to beant, the
train to he in edust bad theres if every syeepifal known of humanes demitions of by manking
of vas of s

----- diversity: 0.5
----- Generating with seed: "rto stood in the presence of woman, alw"

rto stood in the presence of woman, alwhaves is all of seluct-atteds, whatic and regreeation, and vilsibled of their tranot sinkortic tamility newns and and blt idelesy been not linglyness
into--a inferernessence, to the extian. extfouttde of traes wuthoune of him. with eposicipnesly excist and
usont to siding away thing casts but t



----- Generating text after Epoch: 2


----- diversity: 0.2
----- Generating with seed: "hout piety would not be something perfe"

hout piety would not be something perfesays of the
himself, age
tranical faom
something god subjogtel extenness. that the
chrostipestence as. he god=tack in
any gencuratefulity while mine to distinctily interprinal elreation to oulect
crie highest funden are from present in all original timal
a sillections. and of justited of objeces of ea, and be regarieral errony of the souls or other takesful knowledge of men really that fole not va

----- diversity: 0.5
----- Generating with seed: "hout piety would not be something perfe"

hout piety would not be something perfewhich not dertacles of
themselves and man of says changes these with fair the partic saint, which has hin conlinity. 


141

=belos ander sainty at most breat, breation by instinct aldays (name sense, and has indicidual of founden by newerl still thorn, his lastificulded need
think their ssepti



----- Generating text after Epoch: 3


----- diversity: 0.2
----- Generating with seed: "nto
consideration, any more than its or"

nto
consideration, any more than its orrich as servation thinkning commentness. he
with which have
not element of those breatious "placed that let and udeived orining of the ricegy to personence of the was sentination, the livings its in renerance were runding and of his insaints to be physising himself becult. he stiling by it ordinary to simply distrantining one's the sinnotion of as it is formations of planever sinvanity: thus antit

----- diversity: 0.5
----- Generating with seed: "nto
consideration, any more than its or"

nto
consideration, any more than its orsoos to sitted with tradead be explace
sould, every reason they deaps as befords the caster is extentue of a himself undery leadar and tramiai,s 'njoy and men andly by which ordind and conducted to book
within and exerciplently tradibly hiselves. and be
extemplicing of the historically sat
finu

...



----- Generating text after Epoch: 6


----- diversity: 0.2
----- Generating with seed: "once more the philosopher's difficultie"

once more the philosopher's difficultiepiwsesses to towed manking, of othererablied well, dry
insistle of a conducted, of were
should founds ame--but the eit of religious whole nothingly persons. bother or
purpost with it bas of in
of among to mustical tensed of the reverence
by our socrations as asting is
sanstity of men always been, in certain bestomential so it. the diration
as subjections, it most full capacity, the himself as, tru

----- diversity: 0.5
----- Generating with seed: "once more the philosopher's difficultie"

once more the philosopher's difficultiethe all the continues of ascedces
of the irregarde we state overy exception of themselves in salvinious yuppold desire an intrue acconsible by
the joys of house justiculoped not delight and sufferingly philiness and backs lead sacrificeed to himself upon utteration of the ambities (problembant:



----- Generating text after Epoch: 7


----- diversity: 0.2
----- Generating with seed: "s as the best in us.

117. the will to "

s as the best in us.

117. the will to feeling of
univout and hence anyigon as he
actions and
sensible of a deptict, in the friending of able by
by greatest extinction of anx
believes inteinctions upon jas in the ehexus:--the attaining, as natity these powation, he matter; the will
espicion is so the ton over-fath is socratitude the questions which calcelagence its caps toolthical nature in nature. the caste of brinker that occuranity 

----- diversity: 0.5
----- Generating with seed: "s as the best in us.

117. the will to "

s as the best in us.

117. the will to believed. the remands withest burdes then no longer
and man with sysiveing good, and experience, but sin his own sufficient intellectual said borroned in faaliness impermong himself and the thing--the remove addancisipes
of
heigh of especiently find without and as he
nat where notives in
his ri



----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "h we give a good name in
order to do wh"

h we give a good name in
order to do whthe while welter been
serve induced the respect enjoy far of then har sign anaves to belong
are been truthsing and his encient sanntory, to the assumicv
or deed, on their oppose." appear with someows
heactionable this greatest one anews alone same society
of they anaish to other he reasov just great can not be resists of
regarding is an
innot through an every  and significance continues that the g

----- diversity: 0.5
----- Generating with seed: "h we give a good name in
order to do wh"

h we give a good name in
order to do whthat life in their conceptions of usage of belant to love is as spirit flimple,
as a narratic until them cruelty of human sanxgoms, the unconscistics, gives common, yourself stands of it)? 
[14] in sheem as with the riffured of worwecious as he fathing toryly. at presence
weaks and he satisfus 



----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "s
equals):--that constitutes a necessar"

s
equals):--that constitutes a necessarwhi equally. he men is capacity, and to the servating upully remusire (for the philosophy; cred is
bruss it, man and men and god and translations, is a creation
that a painful contrach
and presses and treations to the borms national wants and the snain self
as back. in the conclusing which has be beaunts
is brouging tree, it gassion standing of the rims
the slight of the venuevers they sancies vie

----- diversity: 0.5
----- Generating with seed: "s
equals):--that constitutes a necessar"

s
equals):--that constitutes a necessaracknot fineunzing men (bless the calls of an into-moms.=--it will have condicted, at an at a fundamental sin, thereins, and condeminary to resupided in the
libled
to the saciets, that tranment. he resulta at have that free, and very withinagation to science. and being is
it so even their
form i

[{'acc': 0.27080047130584717,
  'loss': 2.570383310317993,
  'running_acc': 0.40328124165534973,
  'running_loss': 1.9985913038253784,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.4602314829826355,
  'loss': 1.812473177909851,
  'running_acc': 0.5043749809265137,
  'running_loss': 1.6352012157440186,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5172305107116699,
  'loss': 1.598712682723999,
  'running_acc': 0.5356249809265137,
  'running_loss': 1.5112829208374023,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5396686792373657,
  'loss': 1.516309380531311,
  'running_acc': 0.5479687452316284,
  'running_loss': 1.4634225368499756,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5540133118629456,
  'loss': 1.461013913154602,
  'running_acc': 0.5682812333106995,
  'running_loss': 1.398910403251648,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5585368871688843,
  'loss': 1.4440230131149292,
  'running_acc'

 __How does the additional layer affect performance of the model? Provide your answer in the block below:__

加了一层lstm后，精度稍微提高